<a href="https://colab.research.google.com/github/timmyades3/My-oou_csc309_artificial_intelligence/blob/main/notebooks/CSC309_Week05_KRR_Logic_Student_Centred.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CSC309 – Artificial Intelligence  
**Week 5 Lab:** Knowledge Representation & Reasoning — Logic & Inference

**Instructor:** Dr Sakinat Folorunso

**Title:** Associate Professor of AI Systems and FAIR Data **Department:** Computer Sciences, Olabisi Onabanjo University, Ago-Iwoye, Ogun State, Nigeria

**Course Code:** CSC 309

**Mode:** Student‑centred, hands‑on in Google Colab

> Every code cell is commented line‑by‑line so you can follow the logic precisely.

#GROUP MEMBER
1. Adesina Abdulrahman SCI/WD/23/24/0134
2. Adesina samuel SCI/23/24/0145
3. Akinwale victor SCI/23/24/0260,
4. solanke gloria SCI/23/24/1084,
5. meroyi oluwalanumi SCI/23/24/0616,
6. oyedeji elizabeth SCI/23/24/0975,
7. Adeniye Uthman Okikiola Sci/23/24/0122



## How to use this notebook
1. Start with the **Group Log** and **Do Now**.  
2. Run the **Setup** cell once.  
3. Work through **Tasks**. Edit only cells marked **`# TODO(Student)`**.  
4. Use **Quick Checks** to test your understanding.  
5. Finish with the **Reflection**. If you finish early, try the **Extensions**.

In [7]:
#@title 🧑🏽‍🤝‍🧑🏾 Group Log (fill before you start)
# The '#@param' annotations create form fields in Colab for easy input.

group_members = "Adesina Abdulrahman",     "Adesina Samuel",     "Akinwale Victor",     "Solanke Gloria",     "Meroyi Oluwalanumi",     "Oyedeji Elizabeth",     "Adeniye Uthman Okikiola"  #@param {type:"string"}  # Names of teammates
roles_notes = "Driver/Navigator, decisions, questions"  #@param {type:"string"}  # Short working notes

print("👥 Group:", group_members)        # Echo the group list for confirmation
print("📝 Notes:", roles_notes)          # Echo the notes so they're preserved in output

👥 Group: ('Adesina Abdulrahman', 'Adesina Samuel', 'Akinwale Victor', 'Solanke Gloria', 'Meroyi Oluwalanumi', 'Oyedeji Elizabeth', 'Adeniye Uthman Okikiola')
📝 Notes: Driver/Navigator, decisions, questions


### Learning Objectives
- Encode facts/rules in a small **logic knowledge base**.  
- Query using unification; practice NL→logic, CNF/resolution (by hand).

In [8]:
#@title 🔧 Setup
# We try to use 'kanren' for a taste of relational programming.
# If unavailable, we fall back to a tiny custom rule demo.

import sys, subprocess                                     # Tools for optional installs
def pip_install(pkgs):
    for p in pkgs:
        try: __import__(p.split("==")[0])                  # Try importing
        except Exception:
            subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", p])  # Install if missing

try:
    pip_install(["kanren", "unification", "toolz"])        # Attempt to install kanren ecosystem
    from kanren import Relation, facts, run, var           # Import relational primitives
    using_kanren = True                                     # Flag to indicate the backend in use
except Exception as e:
    print("Using simple fallback (kanren not available):", e) # Inform the student
    using_kanren = False                                    # Switch to fallback
print("✅ Setup complete for Week 5.")

✅ Setup complete for Week 5.


In [9]:
#@title 📐 Starter KB (line‑by‑line comments)

if 'using_kanren' in globals() and using_kanren:    # If kanren is available
    parent = Relation()                              # Declare a binary relation 'parent(x,y)'
    facts(parent, ("ada","sam"), ("sam","tayo"), ("sam","ife"), ("ife","bola"))  # Add some facts
    x = var()                                        # Logic variable 'x'
    y = var()                                        # Logic variable 'y'
    # Query: who are Ada's grandchildren?
    result = run(0, x, parent("ada", x), parent(x, y))   # Find x such that parent(ada,x) and parent(x,y)
    print("Grandchildren of Ada (via kanren):", set(result))  # Show solutions
else:
    # Fallback: a tiny Python-based relation
    PARENT = {("ada","sam"), ("sam","tayo"), ("sam","ife"), ("ife","bola")}  # Parent relation as a set of tuples
    def grandchildren(p):                 # Compute grandchildren of 'p' by relational join
        return {c2 for (p1,c1) in PARENT for (p2,c2) in PARENT if p1==p and p2==c1}
    print("Grandchildren of Ada (fallback):", grandchildren("ada"))  # Show result

Grandchildren of Ada (via kanren): {'sam'}


In [10]:
# TODO(Student): Encode 'All humans are mortal; Socrates is human; therefore Socrates is mortal.'
# Option A (kanren): Create relations HUMAN(x) and MORTAL(x); add facts and rules; run a query for Socrates.
# Option B (fallback): Represent as Python sets/dicts and derive 'mortal(socrates)' procedurally.

### Quick Check
Translate **two** everyday constraints into predicate logic, then back into English.  
E.g., “All 300‑level courses have a CA in Week 7.”

In [11]:
# SOLUTION
# OPTION A(KANREN)

from kanren import Relation, facts, run, var

# Declare relations: HUMAN(x) and MORTAL(x)
HUMAN = Relation()   # binary relation used as unary predicate HUMAN(x) by storing tuples (x,)
MORTAL = Relation()  # similarly for MORTAL(x)

# Add fact: Socrates is a human
facts(HUMAN, ("socrates",))    # HUMAN("socrates") is true

# --- Rule (All humans are mortal) ---
# Kanren doesn't automatically add rule-based facts, so we implement a tiny forward-chaining
# step: for every x such that HUMAN(x) we assert MORTAL(x).
x = var()                              # logic variable
humans = run(0, x, HUMAN(x))           # find all humans
# Add corresponding mortal facts
for h in humans:
    facts(MORTAL, (h,))                # derive MORTAL(h)

# Query: is Socrates mortal?
is_mortal = run(0, x, MORTAL("socrates"))   # returns tuple(s) if true
print("Humans found:", set(humans))         # show humans discovered
print("Is Socrates mortal? ->", bool(is_mortal))   # True means mortal fact exists
print("Mortal facts:", set(run(0, x, MORTAL(x))))  # display all mortal facts


Humans found: {'socrates'}
Is Socrates mortal? -> True
Mortal facts: {'socrates'}


In [13]:
# SOLUTION
# OPTION B (FALLBACK)

# Represent unary predicates as Python sets of constants
HUMAN = {"socrates"}   # fact: Socrates is human
MORTAL = set()         # initially empty; we'll derive mortal individuals

# Rule: All humans are mortal
# Implement forward-chaining: for every x in HUMAN, add x to MORTAL
for x in HUMAN:
    MORTAL.add(x)

# Query: is 'socrates' in MORTAL?
print("Humans:", HUMAN)
print("Mortal:", MORTAL)
print("Is Socrates mortal? ->", ("socrates" in MORTAL))



Humans: {'socrates'}
Mortal: {'socrates'}
Is Socrates mortal? -> True
